In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cross_validation import KFold, train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, Imputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

import xgboost as xgb

sns.set_context('poster')
sns.set_style('whitegrid')

np.random.seed(2016)

In [2]:
# load files
train = pd.read_csv('../data/train2016.csv')
test  = pd.read_csv('../data/test2016.csv')
sub   = pd.read_csv('../data/sampleSubmission2016.csv')

In [3]:
# class balance
print('Class Balance\n ', train.Party.value_counts())

Class Balance
  Democrat      2951
Republican    2617
Name: Party, dtype: int64


In [4]:
# missing values
print('Missing values in training set :\n', train.isnull().any().sum())

Missing values in training set :
 106


In [5]:
print('Missing values in test set: \n', test.isnull().any().sum())

Missing values in test set: 
 106


** Since there are lot of missing values in the data, we can carry out an analysis to see what are the different kind of missing values. **

** Most of the features captured are categorical in nature **

In [6]:
def print_unique_values(data):
    categorical_features = data.select_dtypes(include=['object']).columns
    
    for cat in categorical_features:
        print('Number of unique values for feature: %s are: %d'%(cat, data[cat].nunique()))
        
        if data[cat].nunique() < 10:
            print(data[cat].unique())
        else:
            print(data[cat].unique()[:10] + '...')
        
        print('-'*75)
        print('\n')

** As we can see most of the questions have _yes_ / _no_ answers, we can fill missing values appropriately. **

In [7]:
# Methods to get training and test dataset from concatenated data

def get_training_dataset(data):
    mask = data.Party.notnull()
    return data[mask]

def get_test_dataset(data):
    mask = data.Party.isnull()
    return data[mask]

** Relationship between different variables **

```
Checking if two categorical variables are independent can be done with Chi-Squared test of independence.

This is a typical Chi-Square test: 

If we assume that two variables are independent, then the values of the contingency table for these variables should be distributed uniformly. And then we check how far away from uniform the actual values are.
```

```
Let's apply this rule to find out if gender and party are independent or not.

Null Hypothesis: They are independent
Under this hypothesis, we assume their distribution to be uniform.

```

In [8]:
pd.crosstab(train.Party, train.Gender, margins=True)

Gender,Female,Male,All
Party,,,
Democrat,1275,1613,2888
Republican,855,1712,2567
All,2130,3325,5455


In [9]:
pd.crosstab(train.Party, train.Gender, margins=True, normalize=True)

Gender,Female,Male,All
Party,,,
Democrat,0.233731,0.295692,0.529423
Republican,0.156737,0.313841,0.470577
All,0.390467,0.609533,1.000000


In [10]:
cont_table = pd.crosstab(train.Party, train.Gender) 
chi_2, p, dof, expected = sp.stats.chi2_contingency(cont_table.values, correction=False)

In [11]:
print('P-value ', p)

P-value  2.57076096692e-16


`P-value is very close to zero which is enough to discard the hypothesis of independence.`

** Compute Cramer's V**

In [12]:
np.sqrt(chi_2 / (len(cont_table) * min(cont_table.shape[0], cont_table.shape[1])))

4.0959636065625222

` Lower this value, lower the correlation `

** We can say that gender and party are not independent variables. **

** We can perform this type of for all of the categorical variables with the target variable to see whether we can any pair of features that are independent and should not be included in the modelling process as it would add noise to the model. **

In [13]:
def chi_square_test(df, target, candidate_features):
    test_summary = {}
    
    for feat in candidate_features:
        test_summary[(target, feat)] = []
        
        cont_table = pd.crosstab(df[target], df[feat])
        chi2_statistic, p_val, dof, expected = sp.stats.chi2_contingency(cont_table.values, correction=False)
        print('P-value for the pair (%s, %s): %f'%(target, feat, p_val))
        
        test_summary[(target, feat)].append(p_val)
        
        cramer_v = np.sqrt(chi2_statistic / (len(cont_table) * min(cont_table.shape[0], cont_table.shape[1])))
        print('Cramer-V value for pair (%s, %s): %f'%(target, feat, cramer_v))
        
        test_summary[(target, feat)].append(cramer_v)
        print('-'*75)
        print('\n')
        
    return test_summary

In [14]:
def get_candidate_features(df):
    columns = df.select_dtypes(include=['object']).columns
    return columns.drop('Party')

def get_question_features(columns):
    return [col for col in columns if 'Q' in col]

In [15]:
# candidate_features = get_candidate_features(train)
question_features  = get_question_features(train.columns)

In [16]:
test_summary = chi_square_test(train, 'Party', question_features)

P-value for the pair (Party, Q124742): 0.350376
Cramer-V value for pair (Party, Q124742): 0.466930
---------------------------------------------------------------------------


P-value for the pair (Party, Q124122): 0.013485
Cramer-V value for pair (Party, Q124122): 1.235344
---------------------------------------------------------------------------


P-value for the pair (Party, Q123464): 0.390514
Cramer-V value for pair (Party, Q123464): 0.429343
---------------------------------------------------------------------------


P-value for the pair (Party, Q123621): 0.070347
Cramer-V value for pair (Party, Q123621): 0.904835
---------------------------------------------------------------------------


P-value for the pair (Party, Q122769): 0.373139
Cramer-V value for pair (Party, Q122769): 0.445305
---------------------------------------------------------------------------


P-value for the pair (Party, Q122770): 0.343706
Cramer-V value for pair (Party, Q122770): 0.473434
----------------

In [15]:
def select_features_chi2(test_summary, n=10):
    return [k[1] for k, v in sorted(test_summary.items(), key=lambda x: x[1])[:n]]

In [16]:
top_chi2_features = select_features_chi2(test_summary, n=10)

In [17]:
print('Top features selected after chi2 selection are: \n%s'%(top_chi2_features))

Selected Features are: 
['Q109244', 'Q115611', 'Q98197', 'Q113181', 'Q98869', 'Q101163', 'Q99480', 'Q105840', 'Q116881', 'Q120379']


** Preprocessing **

In [18]:
print('Max YOB in the training set', train.YOB.max())
print('Min YOB in the training set ', train.YOB.min())

Max YOB in the training set 2039.0
Min YOB in the training set  1880.0


In [19]:
# combine categorical features
def combine_feature(data, feature1, feature2):
    data[feature1 + '_' + feature2] = data[feature1].astype(str) + data[feature2].astype(str)
    return data

In [20]:
# remove entries with YOB < 1930 and YOB greater than 2000

def remove_entries(data):
    mask = (data.YOB < 1930) & (data.YOB > 2000)
    return data[~mask]

In [21]:
def fill_missing_values(data, selected_features):
    for col in selected_features:
        if data[col].isnull().any():
            data[col] = data[col].fillna(data[col].value_counts().argmax())
    
    return data

def fill_missing_values_with_unique_label(data, selected_features):
    for col in selected_features:
        if data[col].isnull().any():
            data[col] = data[col].fillna('-99999')
            
    return data

def fill_missing_values_with_flag(data, selected_features):
    for col in selected_features:
        if data[col].isnull().any():
            data[col + '_missing'] = data[col].isnull().astype(int)
            data[col] = data[col].fillna(data[col].value_counts().argmax())
    
    return data

In [22]:
def missing_value_features(columns):
    return [col for col in columns if 'missing' in col]

In [33]:
def encode_features(data, selected_features):
    for cat in selected_features:
        if data[cat].dtype == 'object':
            lbl = LabelEncoder()
            lbl.fit(data[cat])
        
            data[cat] = lbl.transform(data[cat])
    
    return data

In [24]:
# concat train and test data-frames
data = pd.concat((train, test))

In [28]:
data['Age'] = 2016 - data['YOB']

In [27]:
features = ['EducationLevel', 'Gender', 'HouseholdStatus', 'Income', 'Age'] + top_chi2_features

In [34]:
data = remove_entries(data)

# fill missing values for only the selected features
data = fill_missing_values_with_unique_label(data, features)
data = encode_features(data, features)

** Datasets **

In [35]:
train_ = get_training_dataset(data)
test_  = get_test_dataset(data)

In [472]:
# add list of features created to mark missing values
# selected_features = top_chi2_features + missing_value_features(train_.columns)

In [36]:
X = train_[features]
y = (train_.Party == 'Democrat').astype(int)

Xtest = test_[features]

In [37]:
Xtr, Xte, ytr, yte = train_test_split(X, y, stratify=y, test_size=0.33, random_state=12128)

** Logistic Regression **

In [38]:
log = LogisticRegression(C=1.)
log.fit(Xtr, ytr)

yhat = log.predict(Xte)
print('Accuracy score on unseen examples ', accuracy_score(yte, yhat))

Accuracy score on unseen examples  0.562568008705


** Random Forest Classifier **

In [39]:
rf = RandomForestClassifier(class_weight='balanced', n_estimators=100, max_depth=5, n_jobs=-1, random_state=1231)
rf.fit(Xtr, ytr)

yhat = rf.predict(Xte)
print('Accuracy score on unseen examples ', accuracy_score(yte, yhat))

Accuracy score on unseen examples  0.607725788901


** Extra Trees Classifier **

In [40]:
etr = ExtraTreesClassifier(class_weight='balanced', n_estimators=150, max_depth=5, n_jobs=-1, random_state=1232)
etr.fit(Xtr, ytr)

yhat = etr.predict(Xte)
print('Accuracy score on unseen examples ', accuracy_score(yte, yhat))

Accuracy score on unseen examples  0.606637649619


** Extreme Gradient Boosting **

In [41]:
xgb_est = xgb.XGBClassifier(n_estimators=100, learning_rate=0.07, min_child_weight=3, seed=21)
xgb_est.fit(Xtr, ytr)

yhat = xgb_est.predict(Xte)
print('Accuracy score on unseen examples ', accuracy_score(yte, yhat))

Accuracy score on unseen examples  0.602285092492


#### Estimating error 

In [42]:
def estimating_error(X, y, est, n_folds=10):
    skf = StratifiedKFold(y, n_folds=n_folds, shuffle=True, random_state=2318)
    cv_score = cross_val_score(est, X, y, scoring='accuracy', cv=skf, n_jobs=-1)
    return cv_score

In [43]:
cv_score_log_10 = estimating_error(Xtr, ytr, log, 10)
print('Mean cross-validation score for Logistic Regression: %f and std: %f '%(np.mean(cv_score_log_10), np.std(cv_score_log_10)))

Mean cross-validation score for Logistic Regression: 0.595152 and std: 0.021785 


In [44]:
cv_score_rf_10 = estimating_error(Xtr, ytr, rf, 10)
print('Mean cross-validation score for Random Forest: %f and std: %f '%(np.mean(cv_score_rf_10), np.std(cv_score_rf_10)))

Mean cross-validation score for Random Forest: 0.637545 and std: 0.026881 


In [45]:
cv_score_etr_10 = estimating_error(Xtr, ytr, etr, 10)
print('Mean cross-validation score for Extra Trees: %f and std: %f '%(np.mean(cv_score_etr_10), np.std(cv_score_etr_10)))

Mean cross-validation score for Extra Trees: 0.635938 and std: 0.026269 


In [46]:
cv_score_xgb_10 = estimating_error(Xtr, ytr, xgb_est, 10)
print('Mean cross-validation score for Extra Trees: %f and std: %f '%(np.mean(cv_score_xgb_10), np.std(cv_score_xgb_10)))

Mean cross-validation score for Extra Trees: 0.645862 and std: 0.022450 


In [47]:
def get_predictions(model, test):
    return model.predict(test)

In [48]:
log_preds = get_predictions(log, Xte)
rf_preds  = get_predictions(rf, Xte)
etr_preds = get_predictions(etr, Xte)
xgb_preds = get_predictions(xgb_est, Xte)

** Analyze our errors **

In [49]:
def get_error_summary(test_df, true, preds, features=['Gender', 'HouseholdStatus']):
    mask = preds != true
    error_summary = {}
    
    gr = test_df[mask].groupby(features).size()
    
    for idx in gr.index.values:
        error_summary[idx] = gr.ix[idx]
    
    return error_summary

In [50]:
log_summary = get_error_summary(Xte, yte, log_preds)
rf_summary = get_error_summary(Xte, yte, rf_preds)
etr_summary = get_error_summary(Xte, yte, etr_preds)
xgb_summary = get_error_summary(Xte, yte, xgb_preds)

In [509]:
log_summary

{(0, 0): 10,
 (0, 1): 7,
 (0, 2): 34,
 (0, 3): 62,
 (0, 4): 138,
 (0, 5): 14,
 (1, 0): 15,
 (1, 1): 6,
 (1, 2): 57,
 (1, 3): 144,
 (1, 4): 231,
 (1, 5): 11}

In [441]:
rf_summary

{(0, 0): 9,
 (0, 1): 7,
 (0, 2): 34,
 (0, 3): 60,
 (0, 4): 134,
 (0, 5): 14,
 (1, 0): 15,
 (1, 1): 6,
 (1, 2): 58,
 (1, 3): 145,
 (1, 4): 241,
 (1, 5): 11}

In [442]:
etr_summary

{(0, 0): 10,
 (0, 1): 7,
 (0, 2): 34,
 (0, 3): 60,
 (0, 4): 137,
 (0, 5): 13,
 (1, 0): 14,
 (1, 1): 5,
 (1, 2): 57,
 (1, 3): 143,
 (1, 4): 239,
 (1, 5): 16}

In [443]:
xgb_summary

{(0, 0): 8,
 (0, 1): 7,
 (0, 2): 33,
 (0, 3): 60,
 (0, 4): 135,
 (0, 5): 13,
 (1, 0): 17,
 (1, 1): 4,
 (1, 2): 59,
 (1, 3): 145,
 (1, 4): 244,
 (1, 5): 13}

** So it is proving to be difficult to get gender type 2 and household type 5 correct in general **

In [444]:
Xte.groupby(['Gender', 'HouseholdStatus']).size()

Gender  HouseholdStatus
0       0                   27
        1                   12
        2                   93
        3                  184
        4                  354
        5                   36
1       0                   42
        1                   10
        2                  132
        3                  339
        4                  577
        5                   32
dtype: int64

** Reason might be that this is the most common pair **

** Lets see whether predictions of these models are correlated or not **

In [51]:
pd.crosstab(log_preds, rf_preds, margins=True, normalize=True)

col_0,0,1,All
row_0,,,
0,0.336235,0.079434,0.415669
1,0.156148,0.428183,0.584331
All,0.492383,0.507617,1.000000


In [52]:
pd.crosstab(etr_preds, rf_preds, margins=True, normalize=True)

col_0,0,1,All
row_0,,,
0,0.454842,0.016866,0.471708
1,0.037541,0.490751,0.528292
All,0.492383,0.507617,1.000000


In [53]:
pd.crosstab(log_preds, etr_preds, margins=True, normalize=True)

col_0,0,1,All
row_0,,,
0,0.321545,0.094124,0.415669
1,0.150163,0.434168,0.584331
All,0.471708,0.528292,1.000000


In [54]:
pd.crosstab(log_preds, xgb_preds, margins=True, normalize=True)

col_0,0,1,All
row_0,,,
0,0.301959,0.113711,0.415669
1,0.124048,0.460283,0.584331
All,0.426007,0.573993,1.000000


In [55]:
pd.crosstab(rf_preds, xgb_preds, margins=True, normalize=True)

col_0,0,1,All
row_0,,,
0,0.410773,0.081610,0.492383
1,0.015234,0.492383,0.507617
All,0.426007,0.573993,1.000000


In [56]:
pd.crosstab(etr_preds, xgb_preds, margins=True, normalize=True)

col_0,0,1,All
row_0,,,
0,0.404244,0.067465,0.471708
1,0.021763,0.506529,0.528292
All,0.426007,0.573993,1.000000


In [57]:
def majority_voting(preds, n_examples, ytest=None):
    final_preds = []
    for i in range(n_examples):
        model_preds = []
        for j in range(len(preds)):
            model_preds.append(preds[j][i])
        
        bin_count = np.bincount(model_preds)
        final_preds.append(np.argmax(bin_count))
    
    if ytest is not None:
        print('Accuracy of majority voting ensemble: %f'%(accuracy_score(ytest, final_preds)))
    return final_preds

In [58]:
_ = majority_voting([log_preds, rf_preds, etr_preds, xgb_preds], len(yte), yte)

Accuracy of majority voting ensemble: 0.607182


#### Full Training

In [59]:
log.fit(X, y)
log_preds = log.predict(Xtest)

rf.fit(X, y)
rf_preds  = rf.predict(Xtest)

etr.fit(X, y)
etr_preds = etr.predict(Xtest)

xgb_est.fit(X, y)
xgb_preds = xgb_est.predict(Xtest)

predictions = majority_voting([log_preds, rf_preds, etr_preds, xgb_preds], len(Xtest))

In [60]:
prediction_labels = map(lambda x: 'Democrat' if x == 1 else 'Republican', predictions)

In [61]:
sub['Predictions'] = list(prediction_labels)

* Logistic Regression - Public : 0.61638	Private: 0.60632
* Ensemble Model ( Logistic, Random Forest, Extra Trees ) - Public : 0.62356	Private : 0.62213
* Ensemble with -999 filled in for missing values - Public: 0.62069 Private: 0.63218
* 10 Questions with other variables - Public: 0.62787 Private: 0.60632

In [62]:
sub.to_csv('../submissions/10_questions.csv', index=False)